# Recommender

In [ ]:
import os

import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from surprise import (
    NMF,
    SVD,
    BaselineOnly,
    CoClustering,
    Dataset,
    KNNBaseline,
    KNNBasic,
    KNNWithMeans,
    KNNWithZScore,
    NormalPredictor,
    Reader,
    SlopeOne,
    SVDpp,
    accuracy,
)
from surprise.model_selection import GridSearchCV, KFold, cross_validate
from transformers import pipeline

from recommender.utils.collaborative import get_Iu, get_Ui

In [ ]:
load_dotenv("../.env")

In [ ]:
MONGO_URI = os.getenv("MONGO_URI")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
INDEX_NAME = "recommender-system"
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {"allow_download": "True"}
model_path = "nlptown/bert-base-multilingual-uncased-sentiment"
RANDOM_STATE = 101
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_EXPERIMENT_NAME = os.getenv("MLFLOW_EXPERIMENT_NAME")

## Reviews Data

In [ ]:
connection = MongoClient(MONGO_URI)
db = connection["shein-mirror"]

In [ ]:
input_data = db["product_reviews"]
data = pd.DataFrame(list(input_data.find()))

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data["rating"].plot(kind="hist")

In [ ]:
data["rating"].value_counts()

In [ ]:
sentiment_pipeline = pipeline(
    "sentiment-analysis", model=model_path, tokenizer=model_path, device="mps", batch_size=8, truncation=True
)

In [ ]:
res = sentiment_pipeline(data["review"].to_list())

In [ ]:
res[0:5]

In [ ]:
data["sentiment"] = [int(r["label"][0:1]) for r in res]
data["sentiment_score"] = [r["score"] for r in res]
data["rating_from_score"] = np.round(data["sentiment_score"] * 5)

In [ ]:
data.head()

In [ ]:
data["sentiment"].plot(kind="hist")

In [ ]:
data["rating_from_score"].plot(kind="hist")

In [ ]:
data.head()

In [ ]:
data.to_parquet("data/processed/reviews.parquet", engine="pyarrow")

## Collaborative Filtering

In [ ]:
data = pd.read_parquet("../data/processed/reviews.parquet", engine="pyarrow")
data.head()

In [ ]:
fig = px.histogram(
    data, x="rating", color="rating", title="Rating distribution - Before sentiment analysis", text_auto=True
)

with open("../img/charts/rating_distribution_before_sentiment.json", "w") as f:
    f.write(fig.to_json())
# fig.to_json("img/charts/rating_distribution_before_sentiment.json")
fig.show()

In [ ]:
data["rating"] = data["sentiment"].apply(lambda x: int(x[0:1]))
data.head()

In [ ]:
fig = px.histogram(
    data, x="rating", color="rating", title="Rating distribution - After sentiment analysis", text_auto=True
)
with open("img/charts/rating_distribution_after_sentiment.json", "w") as f:
    f.write(fig.to_json())
fig.show()

In [ ]:
# db["product_reviews-mirror"].drop()
# db["product_reviews-mirror"].insert_many(data.to_dict(orient="records"))

In [ ]:
data = data[["nickname", "product_id", "rating"]]
data = data.rename(columns={"nickname": "userID", "product_id": "itemID", "sentiment": "rating"})

In [ ]:
data.info()

In [ ]:
users = list(data["userID"].unique())

In [ ]:
len(users)

In [ ]:
products = list(data["itemID"].unique())

In [ ]:
len(products)

In [ ]:
train, test = train_test_split(data, test_size=0.3, random_state=RANDOM_STATE, stratify=data[["rating"]])

In [ ]:
reader = Reader(rating_scale=(1, 5))
data_sp = Dataset.load_from_df(data, reader=reader)
train_sp = Dataset.load_from_df(train, reader=reader)
test_sp = Dataset.load_from_df(test, reader=reader)

In [ ]:
trainset = train_sp.build_full_trainset()
testset = test_sp.build_full_trainset().build_testset()

In [ ]:
algo = [
    SVD(random_state=RANDOM_STATE),
    BaselineOnly(),
    CoClustering(),
    KNNBaseline(),
    KNNWithZScore(),
    KNNWithMeans(),
    SlopeOne(),
    KNNBasic(),
    NormalPredictor(),
    NMF(random_state=RANDOM_STATE),
    SVDpp(random_state=RANDOM_STATE),
]
names = [algo[i].__class__.__name__ for i in range(len(algo))]

In [ ]:
algos = {names[i]: {"algo": algo[i]} for i in range(len(algo))}

In [ ]:
result = np.zeros((len(names), 4))

In [ ]:
with mlflow.start_run(run_name="Default parameters") as run:
    experiment_id = run.info.experiment_id
    for k, v in algos.items():
        with mlflow.start_run(experiment_id=experiment_id, run_name=k, nested=True) as subruns:
            tab = cross_validate(v["algo"], data_sp, cv=5, verbose=True, n_jobs=-1)
            v["test_RMSE"] = tab["test_rmse"]
            v["test_mae"] = tab["test_mae"]
            v["fit_time"] = tab["fit_time"]
            v["test_time"] = tab["test_time"]

            rmse = np.mean(tab["test_rmse"])
            mae = np.mean(tab["test_mae"])
            ft = np.mean(tab["fit_time"])
            tt = np.mean(tab["test_time"])

            mlflow.log_metrics({"rmse_test": rmse, "mae_test": mae, "fit_time": ft, "test_time": tt})
            mlflow.sklearn.log_model(v["algo"], k)

            result[algo.index(v["algo"])] = [rmse, mae, ft, tt]

In [ ]:
plt.figure(figsize=(12, 8))
for k, v in algos.items():
    plt.boxplot(
        v["test_RMSE"],
        positions=[list(algos.keys()).index(k)],
        widths=0.6,
        patch_artist=True,
        boxprops=dict(
            facecolor="C" + str(list(algos.keys()).index(k)), color="C" + str(list(algos.keys()).index(k)), linewidth=2
        ),
    )
plt.ylabel("RMSE")
plt.xticks(range(len(algos)), list(algos.keys()), rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
algos_df = pd.DataFrame(algos)
algos_df = algos_df.T
algos_df.head()

In [ ]:
plt.figure(figsize=(12, 8))
for k, v in algos.items():
    plt.boxplot(
        v["test_mae"],
        positions=[list(algos.keys()).index(k)],
        widths=0.6,
        patch_artist=True,
        boxprops=dict(
            facecolor="C" + str(list(algos.keys()).index(k)), color="C" + str(list(algos.keys()).index(k)), linewidth=2
        ),
    )
plt.ylabel("MAE")
plt.xticks(range(len(algos)), list(algos.keys()), rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
for k, v in algos.items():
    plt.boxplot(
        v["fit_time"],
        positions=[list(algos.keys()).index(k)],
        widths=0.6,
        patch_artist=True,
        boxprops=dict(
            facecolor="C" + str(list(algos.keys()).index(k)), color="C" + str(list(algos.keys()).index(k)), linewidth=2
        ),
    )
plt.ylabel("Time (s)")
plt.xticks(range(len(algos)), list(algos.keys()), rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
for k, v in algos.items():
    plt.boxplot(
        v["test_time"],
        positions=[list(algos.keys()).index(k)],
        widths=0.6,
        patch_artist=True,
        boxprops=dict(
            facecolor="C" + str(list(algos.keys()).index(k)), color="C" + str(list(algos.keys()).index(k)), linewidth=2
        ),
    )
plt.ylabel("Time (s)")
plt.xticks(range(len(algos)), list(algos.keys()), rotation=45)
plt.tight_layout()

In [ ]:
result = result.round(3)
result = pd.DataFrame(result, index=list(algos.keys()), columns=["RMSE", "MAE", "fit_time", "test_time"])
result = result.sort_values(by="RMSE", ascending=False)

In [ ]:
fig = px.imshow(
    result,
    labels=dict(x="Metrics", y="Algorithms", color="Values"),
    x=["RMSE", "MAE", "fit_time", "test_time"],
    y=result.index,
    text_auto=True,
    aspect="auto",
    color_continuous_scale="Blues",
    origin="lower",
    title="Algorithms performance",
)
with open("img/charts/result_metrics.json", "w") as f:
    f.write(fig.to_json())
fig.show()

In [ ]:
param_grid_svd = {
    "n_epochs": [5, 10, 20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6],
    "n_factors": [15, 30, 100],
    "random_state": [RANDOM_STATE],
}

In [ ]:
param_grid_bso = {"bsl_options": {"method": ["als", "sgd"], "n_epochs": [5, 15], "reg_u": [10, 20], "reg_i": [5, 15]}}

In [ ]:
algos = {
    "SVD": {"algo": SVD, "params": param_grid_svd},
    "BaselineOnly": {"algo": BaselineOnly, "params": param_grid_bso},
}

In [ ]:
with mlflow.start_run(run_name="Fine-tuned-selected") as run:
    experiment_id = run.info.experiment_id
    for k, v in algos.items():
        print(k)
        with mlflow.start_run(experiment_id=experiment_id, run_name=k, nested=True) as subruns:
            gs = GridSearchCV(v["algo"], v["params"], measures=["rmse", "mae"], cv=5, n_jobs=-1)
            gs.fit(train_sp)

            mlflow.log_params(gs.best_params["rmse"])
            mlflow.log_metrics({"rmse": gs.best_score["rmse"], "mae": gs.best_score["mae"]})

            algo = v["algo"](**gs.best_params["rmse"])
            algo.fit(trainset)
            predictions = algo.test(testset)
            rmse = accuracy.rmse(predictions)
            mae = accuracy.mae(predictions)

            mlflow.log_metrics({"rmse_test": rmse, "mae_test": mae})

            mlflow.sklearn.log_model(algo, k)

In [ ]:
gs.best_params

In [ ]:
algo.fit(trainset)

In [ ]:
# predictions_train = algo.predict(trainset.build_testset())
predictions_test = algo.test(testset)

In [ ]:
print(accuracy.rmse(predictions_test))

In [ ]:
# Crea la matriz de predicciones
n = len(users)
m = len(products)
recomendation = np.zeros((n, m))

for k in users:
    u = users.index(k)
    for product_id in products:
        i = products.index(product_id)
        recomendation[u, i] = algo.predict(k, product_id, verbose=False)[3]

recomendation = pd.DataFrame(recomendation, index=users, columns=products)

In [ ]:
recomendation.head()

In [ ]:
kf = KFold(n_splits=3)
bsl_options = {"method": "als", "n_epochs": 5, "reg_u": 12, "reg_i": 5}

algo = BaselineOnly(bsl_options=bsl_options)
for trainset, testset in kf.split(data_sp):
    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
algo.fit(data_sp.build_full_trainset())

In [ ]:
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data_sp, measures=["RMSE", "MAE"], cv=3, verbose=False)

In [ ]:
trainset, testset = train_test_split(data_sp, test_size=0.25)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

In [ ]:
df = pd.DataFrame(predictions, columns=["uid", "iid", "rui", "est", "details"])
df["Iu"] = df.uid.apply(get_Iu)
df["Ui"] = df.iid.apply(get_Ui)
df["err"] = abs(df.est - df.rui)

In [ ]:
fig = px.scatter(
    df,
    x="Iu",
    y="err",
    title="Error vs. Number of items rated by user",
    labels={"Iu": "Number of items rated by user", "err": "Error"},
)

with open("img/charts/error_vs_number_of_items_rated_by_user.json", "w") as f:
    f.write(fig.to_json())
fig.show()

In [ ]:
fig = px.histogram(
    df, x="err", title="Error distribution", text_auto=False, labels={"err": "Error"}, histnorm="probability density"
)
with open("img/charts/error_distribution.json", "w") as f:
    f.write(fig.to_json())
fig.show()

In [ ]:
fig = px.line()
fig.add_scatter(
    x=df.groupby("Iu", as_index=False)["err"].min()["Iu"],
    y=df.groupby("Iu", as_index=False)["err"].min()["err"],
    mode="lines",
    name="min",
    line_shape="spline",
)
fig.add_scatter(
    x=df.groupby("Iu", as_index=False)["err"].mean()["Iu"],
    y=df.groupby("Iu", as_index=False)["err"].mean()["err"],
    mode="lines",
    name="mean",
    line_shape="spline",
)
fig.add_scatter(
    x=df.groupby("Iu", as_index=False)["err"].max()["Iu"],
    y=df.groupby("Iu", as_index=False)["err"].max()["err"],
    mode="lines",
    name="max",
    line_shape="spline",
)
fig.layout.title = "Error vs. Number of items rated by user"
fig.layout.xaxis.title = "Number of items rated by user"
fig.layout.yaxis.title = "Error"
fig.layout.legend.title = "Error"

with open("img/charts/error_vs_number_of_items_rated_by_user.json", "w") as f:
    f.write(fig.to_json())
fig.show()